# Article figures

In [1]:
import numpy as np
import scipy.linalg as la
import cvxpy as cvx

import os
import pickle
import plotly.express as px

import hurwitz
import cvx_inference

## Introduction

## Methods

### [X] SCS reduction diagram

In [2]:
n = 5
A = np.array(
    [
        [-2.,0,1.,0,1.],
        [0,-2.,0,1.,0],
        [0,0,-2.,0,1.],
        [-1,0,0,-2.,0],
        [0,-1,0,0,-2.]
     ])
gamma = hurwitz.analytic_gamma(A)
E = np.abs(np.sign(A))

u, U = la.eig(gamma)

Alpha_mat = np.full(((n*(n+1))//2,n**2), 0.0)
beta_vec = np.full(((n*(n+1))//2), 0.0)
zero_inds = np.full((n**2), 0)

c1 = 0
for i in range(n):
    for j in range(i,n):

        if i!=j:
            beta_vec[c1] = 0
        else:
            beta_vec[c1] = -1/(2*u[i]).real
        c2 = 0
        for l in range(n):
            for k in range(n):
                if E[l,k] == 0: zero_inds[c2] = 1
                if (i!=j):
                    Alpha_mat[c1, c2] = cvx_inference.alphas(i, j, l, k, U, u).real
                else:
                    Alpha_mat[c1, c2] = cvx_inference.betas(i, l, k, U).real
                c2 += 1
        c1 += 1

vx = cvx.Variable(n**2)
objective = cvx.Minimize(cvx.norm(cvx.multiply(zero_inds,vx), 1))
constraints = [Alpha_mat@vx == beta_vec]
prob = cvx.Problem(objective, constraints)
probdata, chain, inverse_data = prob.get_problem_data(cvx.SCS)
bbb = probdata["b"].reshape((len(probdata["b"]), 1))
fig = px.imshow(
    probdata["A"].toarray(), 
    color_continuous_midpoint=0.0, 
    color_continuous_scale=px.colors.sequential.RdBu
    )
fig.update_traces(dict(
    
))
fig.update_xaxes(dict(
    showticklabels = False
))
fig.update_yaxes(dict(
    showticklabels = False
))
fig.show()

In [5]:
fig.write_image("figs/SCSmatrix.pdf")

### [X] Example images of reconstruction methods on 10x10

## Results

In [6]:
import experiments
n = 10
experiments.gen_rand_timeseries(Ns = [n], edge_probs=[n*0.1], mats_num=100)

100%|██████████| 100/100 [18:35<00:00, 11.16s/it]


### [ ] Violins of accuracy in different methods

In [2]:
import importlib
importlib.reload(experiments)

NameError: name 'experiments' is not defined

In [7]:
import experiments
n = 10
experiments.run_E_inference(Ns = [n], edge_probs=[n*0.1], mats_num=100, alphas = [0.01, 0.05, 0.1])

100%|██████████| 100/100 [8:59:32<00:00, 323.72s/it]  


### [ ] Plot of accuracy/alignment vs. edge probability

### [ ] An example with external noise ? 

### [ ] Examples with non-linear (saturating) systems
Imagino que para este caso podría ser una figura que muestre ejemplos particulares de la matriz de conectividad, la reconstrucción, y los valores de alineación 